In [ ]:
#|default_exp deck

In [ ]:
#|hide
from nbprocess.showdoc import *

# Deck
> Playing Cards

In [ ]:
#|export
from deck_of_cards.card import Card

In [ ]:
#|export
class Deck:
    """Represents a deck of cards.
    Attributes:
      cards: list of Card objects.
    """
    
    def __init__(self):
        """Initializes the Deck with 52 cards.
        """
        self.cards = []
        for suit in range(4):
            for rank in range(1, 14):
                card = Card(suit, rank)
                self.cards.append(card)

    def __str__(self):
        """Returns a string representation of the deck.
        """
        res = []
        for card in self.cards:
            res.append(str(card))
        return '\n'.join(res)

    def add_card(self, card):
        """Adds a card to the deck.
        card: Card
        """
        self.cards.append(card)

    def remove_card(self, card):
        """Removes a card from the deck or raises exception if it is not there.
        
        card: Card
        """
        self.cards.remove(card)

    def pop_card(self, i=-1):
        """Removes and returns a card from the deck.
        i: index of the card to pop; by default, pops the last card.
        """
        return self.cards.pop(i)

    def shuffle(self):
        """Shuffles the cards in this deck."""
        random.shuffle(self.cards)

    def sort(self):
        """Sorts the cards in ascending order."""
        self.cards.sort()

    def move_cards(self, hand, num):
        """Moves the given number of cards from the deck into the Hand.
        hand: destination Hand object
        num: integer number of cards to move
        """
        for i in range(num):
            hand.add_card(self.pop_card())


A Deck of cards is a collection of `Card` objects:


In [ ]:
deck = Deck()
assert isinstance(deck.pop_card(), Card)

In [ ]:
show_doc(Deck.remove_card)

---

#### Deck.remove_card

> **`Deck.remove_card`**` (self, card)`

Removes a card from the deck or raises exception if it is not there.

card: Card

If we remove a card from the Deck we can verify that it no longer exists:

In [ ]:
card23 = Card(2, 3)
deck.remove_card(card23)

assert card23 not in deck.cards



However, another card that we haven't removed, such as the `10 of hearts` will still be in the Deck of cards because we haven't removed it:


In [ ]:
c = Card(2,10)
assert c in deck.cards
c

10 of Hearts